<a href="https://colab.research.google.com/github/Alfred9/Exploring-LLMs/blob/main/Keyword%20Extraction%20with%20Mistral%207B/Keyword_Extraction_with_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Keyword Extraction with Mistral 7B**





 Keyword extraction is a fundamental task in natural language processing (NLP) that plays a crucial role in understanding and analyzing textual data efficiently. By identifying and extracting the most relevant terms from text, we can streamline information retrieval, summarization, and categorization processes. In this notebook, we will delve into keyword extraction techniques, with a focus on leveraging the Mistral 7B model.

Let's start by installing a number of packages that we are going to use throughout this example:

In [1]:
%%capture
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers
!pip install keybert ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

In [2]:
from ctransformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]


Now, let's proceed with setting up a huggingface Transformers pipeline.

In [3]:
from transformers import AutoTokenizer, pipeline

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# **Prompt Engineering**


Let's see if this works with a very basic example:

In [4]:
response = generator("List 5 interesting facts about Spain?")
print(response[0]["generated_text"])

List 5 interesting facts about Spain?

1. Spain is the fourth most populous country in Europe, with a population of over 46 million people.
2. The official language of Spain is Spanish, also known as Castilian. However, there are several regional


Perfect! It can handle a very basic question. For the purpose of keyword extraction, let's explore whether it can handle a bit more complexity.

In [5]:
prompt = """
I have the following document:
* Successful immunotherapy relies on triggering complex responses involving T cell dynamics in tumors and the periphery. Characterizing these responses remains challenging using static human single-cell atlases or mouse models.

Extract 5 keywords from that document.
"""
response = generator(prompt)
print(response[0]["generated_text"])


I have the following document:
* Successful immunotherapy relies on triggering complex responses involving T cell dynamics in tumors and the periphery. Characterizing these responses remains challenging using static human single-cell atlases or mouse models.

Extract 5 keywords from that document.

**Answer:**
1. Immunotherapy
2. T cells
3. Dynamics
4. Triggering
5. Responses


In [6]:
keyword_prompt = """
[INST]

I have the following document:
- [DOCUMENT]

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST]
"""

In [7]:
prompt = keyword_prompt
print(prompt)


[INST]

I have the following document:
- [DOCUMENT]

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST]



In [8]:
from keybert.llm import TextGeneration
from keybert import KeyLLM

# Load it in KeyLLM
llm = TextGeneration(generator, prompt=prompt)
kw_model = KeyLLM(llm)

In [9]:
documents = [
"Successful immunotherapy relies on triggering complex responses involving T cell dynamics in tumors and the periphery. Characterizing these responses remains challenging using static human single-cell atlases or mouse models."
]

keywords = kw_model.extract_keywords(documents); keywords

[['Successful immunotherapy',
  'T cell dynamics',
  'tumors',
  'periphery',
  'characterizing responses',
  'static human single-cell atlases',
  'mouse models.']]

### Sentence Transformer model

Extracting embeddings of the document using a Sentence Transformer model for keyword extraction

In [10]:
from keybert import KeyLLM
from sentence_transformers import SentenceTransformer

# Extract embeddings
model = SentenceTransformer('BAAI/bge-small-en-v1.5')
embeddings = model.encode(documents, convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Load it in KeyLLM
kw_model = KeyLLM(llm)

# Extract keywords
keywords = kw_model.extract_keywords(documents, embeddings=embeddings, threshold=.5)

In [12]:
keywords

[['Successful immunotherapy',
  'T cell dynamics',
  'tumors',
  'periphery',
  'characterizing responses',
  'static human single-cell atlases',
  'mouse models.']]

In [13]:
from keybert import KeyLLM, KeyBERT

# Load it in KeyBert
kw_model = KeyBERT(llm=llm, model='BAAI/bge-small-en-v1.5')

# Extract keywords
keywords = kw_model.extract_keywords(documents, threshold=.5)

In [14]:
keywords

[['Successful immunotherapy',
  'T cell dynamics',
  'tumors',
  'periphery',
  'characterizing responses',
  'static human single-cell atlases',
  'mouse models.']]